This notebook is to export triplets for each file into a new folder in data. 
- 1 triplet = 1 line 
- filename to match source name 
- filename to 
- deliminator to be | as indicated by kshitj
- for any tags, to use [] 

Why: 
- to be pushed into a file for triplet to text OR for reading, etc. 

example: 
001.txt in folder `docs\triplets_LO`
```
profits | were, buoyed | gains, users
economy | tanked | 10000%
greenspan | boasts | record growth 
...

```

In [ ]:
# !conda install -c conda-forge textacy

In [ ]:
import os 
import spacy 
import textacy  # this is built on top of spacy. it has some out of the box functions like YAKE, but documentation is kinda sucky
from pprint import pprint
from tqdm import tqdm

# Load articles to memory 

In [ ]:
# FOLDER = "./data/BBC/News Articles/business"
FOLDER = "./data/BBC/News Articles_w_CovarianceRes/business" # Coref

# Extract Keywords from Title
def load_articles(PATH):
    news_files = sorted([f for f in os.listdir(PATH) if f.endswith('.txt')])

    news_list = []
    for file in news_files:
        with open(os.path.join(PATH, file), 'r') as f:
            text = f.read()
            news_list.append(text)
    
    return news_list

articles = load_articles(FOLDER)
print(articles[0])

Ad sales boost Time Warner profit

Quarterly profits at Time Warner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.

Time Warner benefited from sales of high-speed internet connections and higher advert sales. Time Warner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Time Warner profit

Quarterly profits at US media giant TimeWarner were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that Time Warner now owns 8% of search-engine Google. But AOL had has mixed fortunes. AOL lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, Time Warner said AOLunderlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. Time Warner hopes to increase subscribers by offering the online service free to Time Warner internet customers and will try to sign up AOLexisting customers 

# Generate Keywords from Title


In [ ]:
##########################################################################################################################################
# HELPER FUNCTIONS
##########################################################################################################################################

def seperate_title_and_body(article, VERBOSE = False): 
    """
    Input: a string file
    Returns: list of format (title, [para1, para2, para3, ...])
    """

    paragraphs = article.split("\n\n")

    title = paragraphs[0]
    body = ' '.join(paragraphs[1:])
    if VERBOSE: 
        print("title:", title)
        print("body:", body)
    
    return (title, body)


# Get Keywords from title 
def get_title_keywords_v1(tup_article, nlp, formatting = "compound", VERBOSE = False):
    """
    =================
    Input
    =================
    tup_article: 
    - a Tuple pair  of format (title, body)
    
    nlp:
    - a spacy nlp pipeline. basic one will suffice 

    formatting: 
    - "atomic" returns lemmatized single words for each compound phrase
    - "compound" returns compound phrase in lowercasing  (default)

    verbose:
    - if True, will show renders and step-wise outputs

    =================
    Returns
    =================
    Returns a tuple pair of string keywords: [[entities],[other keywords]]
    
    =================
    Notes
    =================
    - Entity Keywords are NOT lower cased because this is relevant information
    - Other Keywords are lowercased and lemmatized 

    formatting

    """ 
    # CUSTOMIZE HERE
    title = tup_article[0]
    doc_title = nlp(title)
    tags = ["NOUN", "ADJ", "VERB"]
    entities = ["PERSON", "ORG", "PRODUCT", "EVENT", "GPE", "FAC", "NORP"]
    keywords_ents = []
    keywords_others = []
    processed = []

    if VERBOSE:
        spacy.displacy.render(doc_title, style = "dep")
        spacy.displacy.render(doc_title, style = "ent")
        

    keywords_ents.extend([ent.text for ent in doc_title.ents if ent.label_ in entities])
    
    if VERBOSE:
        print("Keywords (entites):", keywords_ents)

    for tok in doc_title:
        if ((tok.pos_ in tags) 
            and (tok.lemma_ not in processed) 
            and (tok.text not in keywords_others) 
            and (tok.text not in keywords_ents)):
            if tok.dep_ == "compound":
                if formatting == "atomic": 
                    a = tok.lemma_
                    b = tok.head.lemma_
                    keywords_others.append(a)
                    keywords_others.append(b)
                    processed.extend([tok.lemma_, tok.head.lemma_])
                if formatting == "compound": 
                    a = tok.text.lower()
                    b = tok.head.text.lower()
                    keywords_others.append(f"{a} {b}")
                    processed.extend([tok.lemma_, tok.head.lemma_])
                if formatting == "hybrid":  
                    a = tok.text.lower()
                    b = tok.head.text.lower()
                    keywords_others.append(tok.lemma_)
                    keywords_others.append(tok.head.lemma_)
                    keywords_others.append(f"{a} {b}")
                    processed.extend([tok.lemma_, tok.head.lemma_])
            else:
                keywords_others.append(tok.text.lower())

    if VERBOSE:
        print("Keywords (entities + others):", keywords_others)
    return (keywords_ents, keywords_others)
    


##########################################################################################################################################
# Process Sampling
##########################################################################################################################################

# Init pipeline
nlp = spacy.load("en_core_web_lg")
# nlp.add_pipe("merge_entities")  # removed because it doesn't make a noticable difference in ouput

article_keywords = []
for i in range(0,20):
    # print(f'\n=========ARTICLE: {i+1}.txt=========')
    article = seperate_title_and_body(articles[i], VERBOSE = False)
    keywords = get_title_keywords_v1(tup_article = article, nlp = nlp, formatting = "atomic", VERBOSE = False)
    article_keywords.append(keywords)
    print(keywords)

print(f"articles processed: {len(article_keywords)}")


(['Time Warner'], ['ad', 'sale', 'boost', 'profit'])
(['Greenspan'], ['dollar', 'gain', 'speech'])
(['Yukos'], ['unit', 'buyer', 'faces', 'loan', 'claim'])
(['BA'], ['high', 'fuel', 'price', 'hit', 'profits'])
(['Pernod', 'Domecq'], ['takeover', 'talk', 'lifts'])
(['Japan'], ['escapes', 'recession'])
(['US'], ['jobs', 'growth', 'slow'])
(['India'], ['calls', 'fair', 'trade', 'rule'])
(['Ethiopia'], ['crop', 'production', '%'])
(['Court'], ['rejects', '280bn', 'tobacco', 'case'])
([], ['ask', 'tips', 'online', 'ad', 'revival'])
(['Indonesians'], ['face', 'fuel', 'price', 'rise'])
(['Peugeot', 'Mitsubishi'], ['deal', 'boosts'])
(['Telegraph'], ['newspapers', 'axe', 'jobs'])
(['EU'], ['air', 'passenger', 'win', 'new', 'rights'])
(['China'], ['keeps', 'tight', 'rein', 'credit'])
(['Parmalat'], ['boasts', 'doubled', 'profits'])
(['India'], ['rupee', 'hits', 'year', 'high'])
(['India'], ['widens', 'access'])
([], ['call', 'user', 'centre', 'user', "'", 'lose', 'patience'])
articles processed

# Tokenize & Tag body according to Keywords

In [ ]:
##########################################################################################################################################
# HELPER FUNCTIONS
##########################################################################################################################################

def tag_and_tokenize_keywords(doc):

    matcher_ent = spacy.matcher.PhraseMatcher(nlp.vocab, attr="LEMMA")  
    matcher_other = spacy.matcher.PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns_ent = [nlp(keyword) for keyword in keywords[0]]
    patterns_other = [nlp.make_doc(keyword) for keyword in keywords[1]]
    matcher_ent.add("KEYWORD_ENT", patterns_ent)
    matcher_other.add("KEYWORD_OTHER", patterns_other)
    # Create a lemmatized version of the input document
    lemmatized_doc = nlp(" ".join([token.lemma_ for token in doc]))

    # Retokenize the document to merge multi-token keyword spans
    with doc.retokenize() as retokenizer:
        # Iterate through the matchers (matcher_ent, matcher_other) and their target documents (lemmatized_doc, doc)
        for matcher, target_doc in zip([matcher_ent, matcher_other], [lemmatized_doc, doc]):
            # Find matches in the target document using the current matcher
            for match_id, start, end in matcher(target_doc):
                # Create a span in the original document corresponding to the matched keywords
                span = doc[start:end]
                # Merge the span into a single token
                retokenizer.merge(span)

                # Set the 'is_keyword' and 'custom_tag' attributes for the merged token
                span[0].set_extension("is_keyword", default=False, force=True)
                span[0]._.is_keyword = True
                span[0].set_extension("custom_tag", default=None, force=True)
                span[0]._.custom_tag = nlp.vocab.strings[match_id]

    return doc

def add_keywords_to_ents(doc):
    new_ents = []
    for token in doc:
        if token._.is_keyword:
            new_ent = spacy.tokens.Span(doc, token.i, token.i + 1, label=token._.custom_tag)
            new_ents.append(new_ent)

    # Filter out existing entities that overlap with the new keyword entities
    filtered_ents = []
    for ent in doc.ents:
        overlaps = [ent.start <= new_ent.start < ent.end or new_ent.start <= ent.start < new_ent.end for new_ent in new_ents]
        if not any(overlaps):
            filtered_ents.append(ent)

    doc.set_ents(filtered_ents + new_ents)
    return doc

def convert_entities_to_tokens(doc):
    with doc.retokenize() as retokenizer:
        for ent in doc.ents:
            # Create a span for the current entity
            span = doc[ent.start:ent.end]
            # Merge the span into a single token
            retokenizer.merge(span)

    return doc

# Displacy Formatting 
col_highlight1 = "magenta"
col_highlight2 = "yellow"
col_others = "lightblue"
options = {
    "ents": ["KEYWORD_ENT", "KEYWORD_OTHER",
             "ORG", "PRODUCT", "GPE", "LOC", "PERSON", "FAC", "NORP", "DATE", "TIME", "PERCENT", "MONEY", "QUANTITY", "ORDINAL", "CARDINAL", "LANGUAGE", "EVENT", "LAW", "WORK_OF_ART"],
    "colors": {"KEYWORD_ENT": col_highlight1, 
               "KEYWORD_OTHER": col_highlight2, 
               "ORG": col_others, 
               "PRODUCT": col_others, 
               "GPE": col_others, 
               "LOC": col_others, 
               "PERSON": col_others, 
               "FAC": col_others, 
               "NORP": col_others, 
               "DATE": col_others, 
               "TIME": col_others, 
               "PERCENT": col_others, 
               "MONEY": col_others, 
               "QUANTITY": col_others, 
               "ORDINAL": col_others, 
               "CARDINAL": col_others, 
               "LANGUAGE": col_others, 
               "EVENT": col_others, 
               "LAW": col_others, 
               "WORK_OF_ART": col_others}  
}


##########################################################################################################################################
# Process Sampling
##########################################################################################################################################

# Generate Sample
doc_idx = 1
title = seperate_title_and_body(articles[doc_idx])[0]
body = seperate_title_and_body(articles[doc_idx])[1]
keywords = article_keywords[doc_idx]
print("title: ", title)
print("title keywords: ", keywords)
print("body: ", body)

# Create new NLP instances, incl matchers
nlp = spacy.load("en_core_web_lg")

# Pipeline the inputs
doc = nlp(body)
doc = tag_and_tokenize_keywords(doc)
doc = add_keywords_to_ents(doc)
doc = convert_entities_to_tokens(doc)

# spacy.displacy.render(doc, style = "dep")
# spacy.displacy.render(doc, style = "dep", options = options)
spacy.displacy.render(doc, style = "ent", options = options)


title:  Dollar gains on Greenspan speech
title keywords:  (['Greenspan'], ['dollar', 'gain', 'speech'])
body:  Dollar has hit Dollar highest level against the euro in almost three months after Greenspan said the US trade deficit is set to stabilise. And Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce the US trade deficit. In late trading in New York, Dollar reached $1.2871 against the euro , from $1.2974 on Thursday. Market concerns about the US trade deficit has hit Dollar in recent months. On Friday, Greenspan speech sent Dollar higher after Dollar had earlier tumbled on the back of worse-than-expected US jobs data. "I think Greenspan 's taking a much more sanguine view on the US trade deficit than Greenspan's taken for some time," said I New York . "Greenspan's taking a longer-term view, laying out a set of conditions under which the US trade deficit can improve this year and next." Worries about

# Generate Tagged Triplets based on keywords

**Note on SVO function**

The subject_verb_object_triples function in Textacy is designed to extract Subject-Verb-Object (SVO) triples from a given text. It follows a set of heuristics based on dependency parsing and linguistic rules. Here's an explanation of the key steps:

1. Iterate over sentences: The function works on a sentence level, so it first iterates over the sentences in the input text.
2. Initialize verb_sos dictionary: This dictionary stores the subjects and objects associated with each verb in the sentence. The keys in the dictionary are verbs, andthe values are dictionaries containing sets of subjects and objects.
3. Iterate over tokens: For each sentence, the function iterates over its tokens. Each token is analyzed based on its dependency label and part-of-speech tag. The primary purpose of this iteration is to identify the subjects and objects associated with each verb.
4. Identify subjects and objects: The function checks if a token is a subject or an object based on its dependency label. If a token is a subject or object, it is added to the corresponding set in the verb_sos dictionary for the associated verb. Subjects can be nominal (nouns) or clausal (subordinate clauses). Objects can be nominal (nouns), prepositional (introduced by a preposition), or clausal (subordinate clauses).
5. Expand subjects and objects: The function expands subjects and objects to include related tokens, such as compound nouns, conjuncts, or tokens within a clause. This is done using helper functions like expand_noun, expand_verb, and the .subtree attribute of tokens.
6. Handle verb conjuncts: The function addresses cases where multiple verbs are connected by a conjunction (e.g., "I read and wrote"). In such cases, the subjects and objects of the conjunct verbs are shared. The function updates the verb_sos dictionary to ensure that all conjunct verbs have the correct subjects and objects.
7. Generate SVO triples: Finally, the function iterates over the verb_sos dictionary and, for each verb with associated subjects and objects, creates an SVO triple. Each triple is a tuple containing three lists: subjects, verbs, and objects. The subjects, verbs, and objects are sorted by their position in the text (using the .i attribute).

In summary, the function generates SVO triples by analyzing the input text's dependency structure and applying heuristics based on linguistic rules. This approach is effective in many cases but may not always produce perfect results, as it relies on the accuracy of the dependency parser and the underlying linguistic assumptions.

Source code: 
`https://textacy.readthedocs.io/en/latest/_modules/textacy/extract/triples.html#subject_verb_object_triples`

In [ ]:

##########################################################################################################################################
# Helper Functions
##########################################################################################################################################
# The following generates triples regardless of keywords. 
# See ref: https://textacy.readthedocs.io/en/latest/api_reference/extract.html#triples

def generate_triplet2sentence_pairs(doc, arr_keyword, return_type = 'std', keyword_filter = False, VERBOSE = False):
    '''
    Returns a list of tuple-pairs: 
    [(triple, sentence), (triple, sentence), ...]

    Parameters:
    -----------
    doc : spacy.tokens.Doc
        A SpaCy document containing the text to extract SVO triplets from.
    arr_keyword : List[List[str]]
        A nested list of keywords (strings), where each sublist represents a set of related keywords.
    keyword_filter : bool, optional (default=False)
        If True, only include triplets containing at least one keyword from the given keyword list.
    VERBOSE : bool, optional (default=False)
        If True, print additional information, such as the list of flattened keywords.

    Returns:
    --------
    pairs : List[Tuple[str, spacy.tokens.Span]]
        A list of tuple pairs, where each tuple contains a formatted triplet (as a string) and
        the sentence.

    Example usage:
    --------------
    keywords = ['Time Warner', 'sales', 'boost', 'profit']
    svo_pairs = generate_triplet2sentence_pairs(doc, arr_keyword=keywords, keyword_filter=True,
    
    '''
    pairs = []
    combined_data = []
    
    flat_keywords = [item for sublist in arr_keyword for item in sublist]
    if VERBOSE: print("flat keywords: ", flat_keywords)

    # Iterate through sentences in the doc
    for sent in doc.sents:
        # Extract SVO triples for each sentence
        triples = list(textacy.extract.subject_verb_object_triples(sent))
        formatted_triples = []
        for s, v, o in triples:
            # if VERBOSE: 
            #     print("text: ", t[0])
            #     print("type: ", type(t[0]))
            #     print("type: ", type(t[0][0]))
            #     print("lemma: ", [tok.lemma_ for tok in t[0]])
                
            subject = [tok.text.lower() for tok in s]
            verb = [tok.text.lower() for tok in v]
            obj = [tok.text.lower() for tok in o]

            if keyword_filter:
                # Check if any keyword is present in the subject, verb, or object
                if not any(keyword in subject + verb + obj for keyword in flat_keywords):
                    continue
        
            
            formatted_triples = f"{' '.join(subject)} | {' '.join(verb)} | {' '.join(obj)}"
            pairs.append((formatted_triples, sent.text))
            combined_data.append(formatted_triples + " <==> " + sent.text)
            if VERBOSE: print("added: ", formatted_triples)

    if return_type == "std":
        return pairs
    if return_type == "llm":
        return combined_data

##########################################################################################################################################
# Process Sampling
##########################################################################################################################################

keywords = article_keywords[doc_idx]
print(keywords)
pairs = generate_triplet2sentence_pairs(doc, arr_keyword = keywords, keyword_filter = True, VERBOSE= True)
pprint(pairs)

(['Greenspan'], ['dollar', 'gain', 'speech'])
flat keywords:  ['Greenspan', 'dollar', 'gain', 'speech']
added:  dollar | has hit | level
added:  dollar | reached | 1.2871
added:  market concerns | has hit | dollar
added:  greenspan speech | sent | dollar
[('dollar | has hit | level',
  'Dollar has hit Dollar highest level against the euro in almost three months '
  'after Greenspan said the US trade deficit is set to stabilise.'),
 ('dollar | reached | 1.2871',
  'In late trading in New York, Dollar reached $1.2871 against the euro , from '
  '$1.2974 on Thursday.'),
 ('market concerns | has hit | dollar',
  'Market concerns about the US trade deficit has hit Dollar in recent '
  'months.'),
 ('greenspan speech | sent | dollar',
  'On Friday, Greenspan speech sent Dollar higher after Dollar had earlier '
  'tumbled on the back of worse-than-expected US jobs data.')]


# Generate TXT files

In [ ]:
##########################################################################################################################################
# Helper Functions
##########################################################################################################################################

def save_tuple_pairs_to_files(pairs, file1_path, file2_path):
    with open(file1_path, 'w', encoding='utf-8') as file1, open(file2_path, 'w', encoding='utf-8') as file2:
        for a, b in pairs:
            file1.write(f"{a}\n")
            file2.write(f"{b}\n")


## Generic Triplets for Team Use

In [ ]:
##########################################################################################################################################
# Generate Triples and Sentences, orgainzed by filename
# USECASE: for general usage by team
##########################################################################################################################################

# Generate Triples and their Reference Sentences. Save it to a file 
PATH_TRIPLE = 'data/BBC/Generation/Business'
PATH_SENTENCE = './data/BBC/Training/business_sentences/'

article_keywords = []
article_pairs = [] 
for i, article in tqdm(enumerate(articles)):
    
    ####################################
    ## Generate Keywords
    nlp = spacy.load("en_core_web_lg")

    title_body = seperate_title_and_body(article, VERBOSE = False)
    keywords = get_title_keywords_v1(tup_article = title_body, nlp = nlp, formatting = "atomic", VERBOSE = False)
    article_keywords.append(keywords)
    print(keywords)
    
    ####################################
    ## Retokenize Body 

    # Create new NLP instances, incl matchers
    nlp_retok = spacy.load("en_core_web_lg")
    doc = nlp_retok(title_body[1])

    ####################################
    ## Generate Triples
    pairs = generate_triplet2sentence_pairs(doc = doc, arr_keyword = keywords, keyword_filter = True, VERBOSE= False)
    article_pairs.append(pairs)
    filename =f"{i+1:03d}.txt"
    FILE_TRIPLE = PATH_TRIPLE + filename
    FILE_SENTENCE = PATH_SENTENCE + filename

    save_tuple_pairs_to_files(pairs, FILE_TRIPLE, FILE_SENTENCE)


print(f"articles processed: {len(article_keywords)}")


0it [00:00, ?it/s]

(['Time Warner'], ['ad', 'sale', 'boost', 'profit'])


1it [00:03,  3.41s/it]

(['Greenspan'], ['dollar', 'gain', 'speech'])


2it [00:06,  3.15s/it]

...

(['New Year'], ['dollar', 'slides'])


376it [20:55,  3.28s/it]

(['VW', 'Indian'], ['considers', 'opening', 'plant'])


376it [20:59,  3.35s/it]


KeyboardInterrupt: 

## Triplets for TextGen

In [ ]:
##########################################################################################################################################
# Generate FILTERED Triples
# USECASE: for generating summaries
# NOTES: triples are filtered for presence of title keywords in sub, verb, obj. If none, the triple is discarded.
##########################################################################################################################################

# Generate Triples and their Reference Sentences. Save it to a file 
PATH_TRIPLE = './data/BBC/Generation/Business/'
PATH_SENTENCE = './data/BBC/Generation/Temp/'

article_keywords = []
article_pairs = [] 
for i, article in tqdm(enumerate(articles)):

    # print(f'\n=========ARTICLE: {i+1}.txt=========')
    
    ####################################
    ## Generate Keywords
    nlp = spacy.load("en_core_web_lg")

    title_body = seperate_title_and_body(article, VERBOSE = False)
    keywords = get_title_keywords_v1(tup_article = title_body, nlp = nlp, formatting = "atomic", VERBOSE = False)
    article_keywords.append(keywords)
    
    ####################################
    ## Retokenize Body 

    # Create new NLP instances, incl matchers
    nlp_retok = spacy.load("en_core_web_lg")
    doc = nlp_retok(title_body[1])

    ####################################
    ## Generate Triples
    
    pairs = generate_triplet2sentence_pairs(doc = doc, arr_keyword = keywords, keyword_filter = True, VERBOSE= False)
    article_pairs.append(pairs)
    filename =f"{i+1:03d}.txt"
    FILE_TRIPLE = PATH_TRIPLE + filename
    FILE_SENTENCE = PATH_SENTENCE + filename

    save_tuple_pairs_to_files(pairs, FILE_TRIPLE, FILE_SENTENCE)


print(f"articles processed: {len(article_keywords)}")


## Triplets for LLM Tuning


In [ ]:
##########################################################################################################################################
# Generate per-line triples and their reference sentences
# USECASE: for tuning a LLM
# NOTES: for triplet 2 sentence, we return type = "llm". This returns a list of strings, where each string is a triplet + sentence pair
##########################################################################################################################################

# Generate Triples and their Reference Sentences. Save it to a file 

FOLDER = './data/BBC/Training/business_gpt2/'
FILENAME = "business_gpt2.txt"
PATH = FOLDER + FILENAME

article_keywords = []
article_pairs = [] 
training_pairs = [] 
for i, article in tqdm(enumerate(articles)):

    ####################################
    ## Generate Keywords

    nlp = spacy.load("en_core_web_lg")

    title_body = seperate_title_and_body(article, VERBOSE = False)
    keywords = get_title_keywords_v1(tup_article = title_body, nlp = nlp, formatting = "atomic", VERBOSE = False)
    article_keywords.append(keywords)
    print(keywords)
    
    ####################################
    ## Retokenize Body 
    # Create new NLP instances, incl matchers

    nlp_retok = spacy.load("en_core_web_lg")
    doc = nlp_retok(title_body[1])

    ####################################
    ## Generate Triples
    
    pairs = generate_triplet2sentence_pairs(doc = doc, arr_keyword = keywords, return_type = "llm", keyword_filter = False, VERBOSE= False)
    training_pairs.extend(pairs)
    

with open(PATH, 'w', encoding= "utf-8") as file:
    for line in training_pairs:
        file.write(f"{line}\n")
    

print(f"articles processed: {len(article_keywords)}")


0it [00:00, ?it/s]

(['Time Warner'], ['ad', 'sale', 'boost', 'profit'])


1it [00:03,  3.30s/it]

(['Greenspan'], ['dollar', 'gain', 'speech'])


2it [00:06,  3.10s/it]

...

(['BA'], ['high', 'fuel', 'price', 'hit', 'profits'])


4it [00:13,  3.47s/it]

(['Pernod', 'Domecq'], ['takeover', 'talk', 'lifts'])


5it [00:17,  3.45s/it]


KeyboardInterrupt: 